# Capacity Building

## Mixing matrices

By default, the Summer compartmental model assumes that each person in the model comes into contact with every other person at the same rate (homogeneous mixing). This notebook looks at incorporating differences in mixing patterns. For example, children tend to interact more with other children, and less with the elderly (with-like or assortative mixing). When considering the impact of control strategies, especially strategies targeting sub groups of people, it is important to consider these differences in contact patterns.

In [ ]:
# If we are running in google colab, pip install the required packages, 
# but do not modify local environments
try:
  import google.colab
  IN_COLAB = True
  %pip install summerepi
except:
  IN_COLAB = False

In [ ]:
# Python standard library imports come first
from datetime import datetime, timedelta
from typing import List

# Then external package imports
import pandas as pd

# Explicit imports from the summer modelling package
from summer import CompartmentalModel
from summer import Stratification

# Set pandas to use our favourite interactive plotting tool
pd.options.plotting.backend = "plotly"

# Define constants and variables

# Define an arbitrary reference date,
# because we will need numbers (not dates) to go into the model solver
COVID_BASE_DATE = datetime(2019, 12, 31)

# Will need this for indexing the MoH data later
region = "Malaysia"

## Utility functions

Data downloads have been moved to a separate module, which we will now obtain

In [ ]:
# If running in google colab, download the required python module
if IN_COLAB:
    !wget https://raw.githubusercontent.com/monash-emu/AuTuMN/master/notebooks/capacity_building/malaysia/get_mys_data.py

import get_mys_data

In [ ]:
# get mixing matrix data for Malaysia
if IN_COLAB:
    !wget https://raw.githubusercontent.com/monash-emu/AuTuMN/master/notebooks/capacity_building/malaysia/MYS_matrices.pkl

mixing_matrix = pd.read_pickle("MYS_matrices.pkl", compression='infer')
age_mixing_matrix = mixing_matrix["all_locations"]

## Get data
Now call the functions from our imported module to obtain the required data


In [ ]:
df = get_mys_data.fetch_mys_data()
initial_population = get_mys_data.get_initial_population(region)
observations = get_mys_data.get_target_observations(df, region, "cases")

In [ ]:
start_date = datetime(2021, 1, 1)  # Define the model's start date
end_date = start_date + timedelta(days=300)  # Define the model's duration

# Numeric representation of the start and end dates
start_date_int = (start_date - COVID_BASE_DATE).days
end_date_int = (end_date - COVID_BASE_DATE).days

In [ ]:
# Define a target set of observations to compare against our modelled outputs later
notifications_target = observations[start_date: end_date]["cases_new"]

## Model

### Define a model

In [ ]:
def build_unstratified_model(base_compartments: List[str], parameters: dict) -> CompartmentalModel:
    """
    Create a compartmental model, with the minimal compartmental structure needed to run and produce some sort of 
    meaningful outputs.
    
    Args:
        base_compartments: The names of the base (unstratified compartments)
        infectious_seed: The number of infectious persons to start the epidemic
        parameters: Flow parameters
    
    """

    model = CompartmentalModel(
        times=(parameters["start_time"], parameters["end_time"]),
        compartments=base_compartments,
        infectious_compartments=["I"],
        ref_date=COVID_BASE_DATE
    )

    infectious_seed = parameters["infectious_seed"]

    model.set_initial_population(distribution={"S": initial_population - infectious_seed, "E": 0, "I": infectious_seed})
    
    # Susceptible people can get infected
    model.add_infection_frequency_flow(
        name="infection", 
        contact_rate=parameters["contact_rate"], 
        source="S", 
        dest="E"
    )
    # Expose people transition to infected
    model.add_transition_flow(
        name="progression",
        fractional_rate=parameters["progression_rate"],
        source="E",
        dest="I",
    )

    # Infectious people recover
    model.add_transition_flow(
        name="recovery",
        fractional_rate=parameters["recovery_rate"],
        source="I",
        dest="R",
    )

    # Add an infection-specific death flow to the I compartment
    model.add_death_flow(name="infection_death", death_rate=parameters["death_rate"], source="I")

    # We will also request an output for the 'progression' flow
    model.request_output_for_flow("progressions", "progression")

    return model

## Stratifications



In [ ]:
def get_age_stratification(compartments_to_stratify: List[str], age_mixing_matrix) -> Stratification:
    """
    Create a fairly simple summer stratification object that splits all compartments
    to 16 age groups from 0-75years.
    
    Args:
        compartments_to_stratify: List of the compartments to stratify, which should be all the compartments
    Returns:
        A summer stratification object to represent age stratification (not yet applied)
    """
    
    # Create the stratification
    strata = [str(5 * i) for i in range(16)]
    strat = Stratification(name="age", strata=strata, compartments=compartments_to_stratify)

    
    # Split the population so that >60 age groups consists of 60% of the population
    pop_split = dict.fromkeys(strata, 0.033)
    pop_split.update(dict.fromkeys(['60', '65','70','75'], 0.15))
    strat.set_population_split(pop_split)
    
    # Add the mixing matrix to the stratification
    strat.set_mixing_matrix(age_mixing_matrix)

    return strat

In [ ]:
def build_stratified_model(base_compartments: List[str], parameters: dict, age_mixng_matrix) -> CompartmentalModel:
    model = build_unstratified_model(base_compartments, parameters)

    age_strat = get_age_stratification(base_compartments, age_mixing_matrix)

    # Apply the stratifications we developed previously
    model.stratify_with(age_strat)

    return model

# Outputs

In [ ]:
# Build and run the stratified model
parameters = {
    "contact_rate": 10.,
    "progression_rate": 0.2,
    "recovery_rate": 0.2,
    "death_rate": 0.,
    "reporting_fraction": 0.01,
    "start_time": start_date_int,
    "end_time": end_date_int,
    "infectious_seed": 100.
}
base_compartments = ["S", "E", "I", "R"]

model = build_stratified_model(base_compartments, parameters, age_mixing_matrix)
model.run()

In [ ]:
progressions = model.get_derived_outputs_df()["progressions"]
notifications_modelled = progressions * parameters["reporting_fraction"]

In [ ]:
pd.DataFrame(
    {
        "observed": notifications_target,
        "modelled": notifications_modelled
    }
).plot()

# Manually calibrating to fit to notifications of Delta Wave

In [ ]:
# Define the model integration time to capture the Delta wave
start_date = datetime(2021, 3, 20)

model_duration = 290
end_date = start_date + timedelta(days=model_duration)

# Convert our start and end times to a numeric representation
start_date_int = (start_date - COVID_BASE_DATE).days
end_date_int = (end_date - COVID_BASE_DATE).days

# Define a set of parameters for this run
custom_params = {
        "contact_rate": 0.43,
        "progression_rate": 0.2,
        "recovery_rate": 0.2,
        "death_rate": 0.01,
        "reporting_fraction": 0.01,
        "start_time": start_date_int,
        "end_time": end_date_int,
        "infectious_seed": 100.
    }

# Run using our custom parameters
model = build_stratified_model(base_compartments, custom_params, age_mixing_matrix)
model.run()

# Get the outputs
progressions = model.get_derived_outputs_df()["progressions"]
notifications_modelled = progressions * custom_params["reporting_fraction"]

notifications_target = observations[start_date: end_date]["cases_new"]

pd.DataFrame(
    {
        "observed": notifications_target,
        "modelled": notifications_modelled
    }
).plot()